In [1]:
import httpx
import asyncio
import json
from bs4 import BeautifulSoup
import polars as pl
from tqdm import tqdm
import time
import os
import random

# Configuration
max_pages = [2, 5, 10, 20, 15]  # Maximum number of pages for each level
base_url = "https://www.fortiguard.com/encyclopedia?type=ips&risk={level}&page={i}"

# Store skipped pages
skipped_pages = {}

# Asynchronous function to fetch the HTML content with retries and backoff
async def fetch_page(client, url, retries=3, delay=2):
    try:
        response = await client.get(url, timeout=10.0)
        response.raise_for_status()  # Ensure we throw an error on bad status codes
        return response.text
    except httpx.RequestError as e:
        print(f"Error fetching {url}: {e}")
        if retries > 0:
            print(f"Retrying {url} in {delay} seconds...")
            await asyncio.sleep(delay)  # Wait before retrying
            return await fetch_page(client, url, retries - 1, delay * 2)  # Exponentially increase delay
        else:
            # Log the error into a file
            with open('error_log.txt', 'a') as log_file:
                log_file.write(f"Failed to fetch {url} after retries\n")
            return None

# Function to scrape data from a single page
async def scrape_page(client, level, page_number):
    url = base_url.format(level=level, i=page_number)
    html = await fetch_page(client, url)
    if html is None:
        return None

    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    data = []

    # Find the articles (modify this as per actual HTML structure)
    articles = soup.find_all('article')  # Adjust this based on actual HTML tags
    for article in articles:
        title = article.find('h2').get_text(strip=True)  # Adjust as per actual tag
        link = article.find('a')['href']
        data.append({'title': title, 'link': link})

    return data

# Main scraping function
async def scrape_level(level, max_pages):
    async with httpx.AsyncClient() as client:
        all_data = []
        for page in range(1, max_pages[level-1] + 1):
            data = await scrape_page(client, level, page)
            if data:
                all_data.extend(data)
            else:
                if level not in skipped_pages:
                    skipped_pages[level] = []
                skipped_pages[level].append(page)

            # Add a random delay between requests to avoid rate limiting
            await asyncio.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds

        # Create a Polars DataFrame
        df = pl.DataFrame(all_data)

        # Save as CSV file
        df.write_csv(f"datasets/forti_lists_{level}.csv")

# Main function to coordinate the scraping
async def main():
    # Create datasets directory if not exists
    if not os.path.exists("datasets"):
        os.makedirs("datasets")

    tasks = []
    for level in range(1, 6):
        tasks.append(scrape_level(level, max_pages))

    # Run all tasks concurrently
    await asyncio.gather(*tasks)

    # Save skipped pages as JSON
    if skipped_pages:
        with open('datasets/skipped.json', 'w') as f:
            json.dump(skipped_pages, f, indent=4)

# Entry point for Jupyter / Interactive environments
if __name__ == '__main__':
    start_time = time.time()
    await main()  # Directly use await instead of asyncio.run()
    print(f"Scraping completed in {time.time() - start_time:.2f} seconds.")

Scraping completed in 139.48 seconds.
